In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import time

from bs4 import BeautifulSoup
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans

# Load saved model from part II

In [2]:
model = Word2Vec.load("300features_40minwords_10context")
print type(model.syn0)
print model.syn0.shape

<type 'numpy.ndarray'>
(16490, 300)


# Load train data

In [3]:
# Read data from files 
train = pd.read_csv( "labeledTrainData.tsv", header = 0, delimiter = "\t", quoting = 3 )
test = pd.read_csv( "testData.tsv", header = 0, delimiter = "\t", quoting = 3 )

# Verify the number of reviews that were read (100,000 in total)
print "Read %d labeled train reviews and %d labeled test reviews\n" % \
      (train["review"].size, test["review"].size)

Read 25000 labeled train reviews and 25000 labeled test reviews



# Recall model parameters

In [4]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 3       # Number of threads to run in parallel
context = 10          # Context window size                   
downsampling = 1e-3   # Downsample setting for frequent words

# Helper function to split review into words from part II

In [5]:
def review_to_wordlist( review, remove_stopwords = False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters (optional)
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(nltk.corpus.stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

# Helper functions to average word vectors

In [6]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,), dtype = "float32")
    #
    nwords = 0.0
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.0
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords) 
    return featureVec

In [7]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews), num_features), dtype = "float32")
    # 
    # Loop through the reviews
    for review in reviews:
        #
        # Print a status message every 1000th review
        if counter%1000. == 0.:
            print "Review %d of %d" % (counter, len(reviews))
        # 
        # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        #
        # Increment the counter
        counter = counter + 1.
    return reviewFeatureVecs

# Create feature vector for each word and average them over reviews

In [8]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, remove_stopwords = True ))

print "Creating average feature vecs for train reviews"
trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

Creating average feature vecs for train reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000


In [9]:
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, remove_stopwords = True ))

print "Creating average feature vecs for test reviews"
testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000


# Random Forest

### Train the model

In [10]:
# Fit a random forest to the training data, using 100 trees

forest = RandomForestClassifier( n_estimators = 100 )

print "Fitting a random forest to labeled training data..."
forest = forest.fit( trainDataVecs, train["sentiment"] )

Fitting a random forest to labeled training data...


### Predict test data output

In [23]:
# Test & extract results 
result = forest.predict( testDataVecs )

### Write prediction to file

In [24]:
## Write the test results 
output = pd.DataFrame( data = {"id" : test["id"], "sentiment" : result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index = False, quoting = 3 )

We found that this produced results much better than chance, but underperformed Bag of Words by a few percentage points.

Since the element-wise average of the vectors didn't produce spectacular results, perhaps we could do it in a more intelligent way? A standard way of weighting word vectors is to apply ["tf-idf"](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) weights, which measure how important a given word is within a given set of documents. One way to extract tf-idf weights in Python is by using scikit-learn's [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html), which has an interface similar to the CountVectorizer that we used in Part 1. However, when we tried weighting our word vectors in this way, we found no substantial improvement in performance.


### Predict probabilities

In [13]:
# Test & extract results 
result = forest.predict_proba( testDataVecs )
result = [x[1] for x in result]

In [14]:
## Write the test results 
output = pd.DataFrame( data = {"id" : test["id"], "sentiment" : result} )
output.to_csv( "Word2Vec_AverageVectors_proba.csv", index = False, quoting = 3 )

# Clustering

In K-Means, the one parameter we need to set is "K," or the number of clusters. How should we decide how many clusters to create? Trial and error suggested that small clusters, with an average of only 5 words or so per cluster, gave better results than large clusters with many words.

K-Means clustering with large K can be very slow; the following code took 75 minutes on my computer. Below, we set a timer around the K-Means function to see how long it takes.

### Cluster all words in the vocabulary based on their feature vectors

In [16]:
start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.syn0
num_clusters = word_vectors.shape[0] / 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print "Time taken for K Means clustering: ", elapsed, "seconds."

Time taken for K Means clustering:  4348.60536695 seconds.


### Create dictionary {word : cluster_number}

The cluster assignment for each word is now stored in idx, and the vocabulary from our original Word2Vec model is still stored in model.index2word. For convenience, we zip these into one dictionary as follows:

In [15]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.index2word, idx ))

NameError: name 'idx' is not defined

In [31]:
print len(word_centroid_map)
word_centroid_map['flower']

16490


2920

### Explore the clusters
Print words from the first ten clusters.

In [32]:
# For the first 10 clusters
for cluster in xrange(0,10):
    #
    # Print the cluster number  
    print "\nCluster %d" % cluster
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        if( word_centroid_map.values()[i] == cluster ):
            words.append(word_centroid_map.keys()[i])
    print words


Cluster 0
[u'talk', u'talking']

Cluster 1
[u'workout']

Cluster 2
[u'garth', u'logan']

Cluster 3
[u'parallels', u'dynamics', u'rivalry', u'clash', u'tensions', u'conflicts']

Cluster 4
[u'iago', u'gibbs']

Cluster 5
[u'troma']

Cluster 6
[u'fashionable', u'improving']

Cluster 7
[u'psychiatric', u'inadvertently', u'unknowingly', u'creasy', u'hobby', u'gathers', u'recovered', u'comrades', u'humiliating', u'fingerprints', u'conducting', u'awakened', u'beetle', u'awaits', u'protecting', u'abandoning', u'mount', u'instructions', u'courts']

Cluster 8
[u'existing', u'unrelated', u'excluding', u'defects', u'untold', u'linking']

Cluster 9
[u'smacks']


### Helper function to create bag of centroids
Function takes list of words from a single review and calculates bag vector where each component stands for a number of words from a certain cluster.

In [33]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype = "float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

### Create bag of centroids for train and test data

In [34]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["review"].size, num_clusters), dtype = "float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    if counter%1000.0 == 0.0:
        print "Review %d of %d" % (counter, len(clean_train_reviews))
    train_centroids[counter] = create_bag_of_centroids( review, word_centroid_map )
    counter += 1

Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000


In [35]:
# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), dtype = "float32" )

counter = 0
for review in clean_test_reviews:
    if counter%1000.0 == 0.0:
        print "Review %d of %d" % (counter, len(clean_test_reviews))
    test_centroids[counter] = create_bag_of_centroids( review, word_centroid_map )
    counter += 1

Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000


### Fit Random Forest on bag cluster centroids

In [36]:
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print "Fitting a random forest to labeled training data..."
forest = forest.fit(train_centroids,train["sentiment"])

Fitting a random forest to labeled training data...


### Predict test data output

In [37]:
result = forest.predict(test_centroids)

### Write prediction to file

In [38]:
# Write the test results 
output = pd.DataFrame(data = {"id" : test["id"], "sentiment" : result})
output.to_csv( "BagOfCentroids.csv", index = False, quoting = 3 )

# Deep Learning in Python packages
- **Theano** offers very low-level, nuts and bolts functionality for building deep learning systems. You can also find some good tutorials on their site.
- **Caffe** is a deep learning framework out of the Berkeley Vision and Learning Center.
- **Pylearn2** wraps Theano and seems slightly more user friendly.
- **OverFeat** was used to win the Kaggle Cats and Dogs competition.